In [46]:
import numpy as np
from calculate_tissue_mask import calculate_tissue_mask 
import pickle
from PIL import Image
# Disable the maximum image pixel check
Image.MAX_IMAGE_PIXELS = None


In [47]:
#Load WS (example)
datafile = r'\\10.99.68.52\Kiemendata\Valentina Matos\coda to python\test model\04_19_2024\net.pkl'
with open(datafile, 'rb') as f:
    data = pickle.load(f)
WS = data['WS']
print(f'WS: {WS}')

WS: [[2, 0, 0, 1, 0, 0, 2, 0, 2, 2, 2, 0, 0], [7, 6], [1, 2, 3, 4, 5, 6, 7, 8, 7, 9, 10, 8, 11], [6, 5, 4, 11, 1, 2, 3, 8, 10, 12, 13, 7, 9], []]


In [48]:
#inputs
imnm = 'SG_013_0061'
pth = r'\\10.99.68.52\Kiemendata\Valentina Matos\coda to python\test model\5x'
I0, TA, _ = calculate_tissue_mask(pth, imnm) 
model_name = '02_23_2024'
numclass=max(WS[3])
print(f'numclass: {numclass}')

Existing TA loaded
numclass: 13


In [49]:
#________________________Start of the function___________________
import os
from PIL import Image
import numpy as np
from skimage import morphology, measure
from scipy import ndimage
from skimage.measure import label


In [50]:
#Should be part of load_annotation_data:
pth = r'\\10.99.68.52\Kiemendata\Valentina Matos\coda to python\test model'
outpth = os.path.join(pth,'data',imnm,'')
print(f'outpth: {outpth}')


outpth: \\10.99.68.52\Kiemendata\Valentina Matos\coda to python\test model\data\SG_013_0061\


In [52]:
print('4. of 4. Creating bounding box tiles of all annotations')
try:
    imlabel = np.array(Image.open(os.path.join(outpth, 'view_annotations.tif')))
except:
    imlabel = np.array(Image.open(os.path.join(outpth, 'view_annotations_raw.tif')))

4. of 4. Creating bounding box tiles of all annotations


In [66]:
# Create directories:
pthbb = os.path.join(outpth, model_name + '_boundbox')
pthim = os.path.join(pthbb, 'im')
pthlabel = os.path.join(pthbb, 'label')

if os.path.isdir(pthim):
    os.rmdir(pthim)
if os.path.isdir(pthlabel):
    os.rmdir(pthlabel)

os.makedirs(pthim)
os.makedirs(pthlabel)

In [67]:
#Image Processing
# Perform morphological closing
tmp = ndimage.binary_closing(imlabel > 0, structure=morphology.disk(10))
# Fill holes in the binary image
tmp = ndimage.binary_fill_holes(tmp)
# Remove small objects with less than 300 pixels
tmp = morphology.remove_small_objects(tmp, min_size=300)

In [68]:
L = label(tmp)
numann = np.zeros((np.max(L) + 1, numclass))

In [69]:
for pk in range(1, np.max(L) + 1):
    
    # Create a binary mask for the current component
    tmp = (L == pk).astype(float)
    a = np.sum(tmp, axis=1)
    b = np.sum(tmp, axis=0)
    # rect = [np.argmax(a > 0), np.argmax(a > 0), np.argmax(b > 0), np.argmax(b > 0)]
    rect = [np.nonzero(b)[0][0], np.nonzero(b)[0][-1],np.nonzero(a)[0][0], np.nonzero(a)[0][-1]]
    # rect = [np.argmax(a), np.argmax(a[::-1]), np.argmax(b), np.argmax(b[::-1])]
    # rect = [np.argmax(a > 0), np.argmax(np.flip(a) > 0), np.argmax(b > 0), np.argmax(np.flip(b) > 0)]
    
    # Crop the binary mask to the region defined by the bounding box
    tmp = tmp[rect[2]:rect[3]+1, rect[0]:rect[1]+1]

    # make label and image bounding boxes
    tmplabel = imlabel[rect[2]:rect[3]+1, rect[0]:rect[1]+1] * tmp
    tmpim = I0[rect[2]:rect[3]+1, rect[0]:rect[1]+1, :]

    nm = str(pk).zfill(5)
    Image.fromarray(tmpim.astype(np.uint8)).save(os.path.join(pthim, f'{nm}.tif'))
    Image.fromarray(tmplabel.astype(np.uint8)).save(os.path.join(pthlabel, f'{nm}.tif'))

    for anns in range(numclass):
        numann[pk, anns] = np.sum(tmplabel == anns)

In [70]:
ctlist = [f for f in os.listdir(pthim) if f.endswith('.tif')]
bb = 1  # indicate that xml file is fully analyzed

In [71]:
annotations_file = os.path.join(outpth, 'annotations.pkl')
data = {'numann': numann, 'ctlist': ctlist, 'bb': bb}

In [73]:
if os.path.join(outpth, 'annotations.pkl'):
    with open(annotations_file, 'ab') as f: #append new data if the file already exists
        pickle.dump(data, f)
else:
    with open(annotations_file, 'wb') as f: #save data to new file 'write binary mode'
        pickle.dump(data, f)

In [77]:
from PIL import Image
import os

def images_equal(image1_path, image2_path):
    """
    Compare two images to check if they are equal.

    Parameters:
        image1_path (str): Path to the first image.
        image2_path (str): Path to the second image.

    Returns:
        bool: True if the images are equal, False otherwise.
    """
    image1 = Image.open(image1_path)
    image2 = Image.open(image2_path)
    
    # Check if images have the same size and mode
    if image1.size != image2.size or image1.mode != image2.mode:
        return False
    
    # Compare pixel values
    pixels1 = list(image1.getdata())
    pixels2 = list(image2.getdata())
    if pixels1 != pixels2:
        return False
    
    return True

# Usage example:
pth = r"\\10.99.68.52\Kiemendata\Valentina Matos\coda to python\test model" 
image_path_python = os.path.join(pth, 'data python', 'SG_013_0061', '02_23_2024_boundbox')
image_path_matlab = os.path.join(pth, 'data matlab', 'SG_013_0061', '02_23_2024_boundbox')

impython_path = os.path.join(image_path_python, 'im', '00001.tif')
immaltab_path = os.path.join(image_path_matlab, 'im', '00001.tif')
imlabelpython_path = os.path.join(image_path_python, 'label', '00001.tif')
imlabelmatlab_path = os.path.join(image_path_matlab, 'label', '00001.tif')

# Test if images are equal
are_images_equal = images_equal(impython_path, immaltab_path)
print("Are the images equal?", are_images_equal)


Are the images equal? True


In [75]:
# # Unit test if generated images are the same as the MATLAB 
# import unittest
# from PIL import Image
# import os
# 
# def images_equal(image1_path, image2_path):
#     """
#     Compare two images to check if they are equal.
# 
#     Parameters:
#         image1_path (str): Path to the first image.
#         image2_path (str): Path to the second image.
# 
#     Returns:
#         bool: True if the images are equal, False otherwise.
#     """
#     image1 = Image.open(image1_path)
#     image2 = Image.open(image2_path)
#     
#     # Check if images have the same size and mode
#     if image1.size != image2.size or image1.mode != image2.mode:
#         return False
#     
#     # Compare pixel values
#     pixels1 = list(image1.getdata())
#     pixels2 = list(image2.getdata())
#     if pixels1 != pixels2:
#         return False
#     
#     return True
# 
# class TestImageEquality(unittest.TestCase):
#     def setUp(self):
#         pth = r"\\10.99.68.52\Kiemendata\Valentina Matos\coda to python\test model" 
#         image_path_python = os.path.join(pth, 'data python', 'SG_013_0061', '02_23_2024_boundbox')
#         image_path_matlab = os.path.join(pth, 'data matlab', 'SG_013_0061', '02_23_2024_boundbox')
# 
#         self.impython_path = os.path.join(image_path_python, 'im', '00001.tif')
#         self.immaltab_path = os.path.join(image_path_matlab, 'im', '00001.tif')
#         self.imlabelpython_path = os.path.join(image_path_python, 'label', '00001.tif')
#         self.imlabelmatlab_path = os.path.join(image_path_matlab, 'label', '00001.tif')
# 
#     def test_images_equal(self):
#         are_images_equal = images_equal(self.impython_path, self.immaltab_path)
#         self.assertTrue(are_images_equal, "Images are not equal")
# 
# if __name__ == '__main__':
#     unittest.main()


E
ERROR: C:\Users\Valentina\AppData\Roaming\jupyter\runtime\kernel-0139b683-d7c6-4caf-bfe5-494d7ea33eb3 (unittest.loader._FailedTest)
----------------------------------------------------------------------
AttributeError: module '__main__' has no attribute 'C:\Users\Valentina\AppData\Roaming\jupyter\runtime\kernel-0139b683-d7c6-4caf-bfe5-494d7ea33eb3'

----------------------------------------------------------------------
Ran 1 test in 0.000s

FAILED (errors=1)


SystemExit: True

C:\Users\Valentina\miniconda3\envs\CODA_python\lib\site-packages\IPython\core\interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
